# Radial  Basis Function

## import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## RBF
$$ \phi (x) = \sum_{i=1}^q \omega_i \rho (x_i, c_i) + b $$
其中：

$q$为隐层神经元个数

$c_i$和$\omega_i$分别为第$i$个隐层神经元所对应的中心和权重

$\rho (x_i, c_i)$是径向基函数，这是某种沿径向对称的标量函数，通常定义为样本$x$到数据中心$c_i$之间的欧式距离的单调函数，常用的高斯径向基函数形如
$$ \rho (x_i, c_i) = e^{-\beta_i \lVert x - c_i \rVert^2} $$

### BP偏导数公式
这里使用均方根误差作为损失函数的RBF神经网络。
$$ \frac{\partial E}{\partial \omega_i} = [y - \phi (x)] \rho (x_i, c_i) $$
$$ \frac{\partial E}{\partial \beta} = [y - \phi (x)]\cdot \omega_i \cdot \rho (x_i, c_i) \cdot (-\lVert x - c_i \rVert^2) $$
$$ \frac{\partial E}{\partial b} = y - \phi (x) $$




In [2]:
def RBF_forward(X_, parameters_):
    m, n = X_.shape
    beta = parameters_['beta']
    W = parameters_['W']
    c = parameters_['c']
    b = parameters_['b']

    t_ = c.shape[0]
    p = np.zeros((m, t_))  # 中间隐藏层的激活值     对应高斯径向基函数
    x_c = np.zeros((m, t_))  # 高斯径向基函数中 x - c_{i}
    for i in range(t_):
        x_c[:, i] = np.linalg.norm(X_ - c[[i],], axis=1) ** 2

        p[:, i] = np.exp(-beta[0, i] * x_c[:, i])

    a = np.dot(p, W.T) + b
    return a, p, x_c

def RBF_backward(a_, y_, x_c, p_, parameters_):
    m, n = a_.shape
    grad = {}
    beta = parameters_['beta']
    W = parameters_['W']

    da = (a_ - y_)      # 损失函数对输出层的偏导 ，这里的a其实对应着  输出层的y_hat

    dw = np.dot(da.T, p_) / m
    db = np.sum(da, axis=0, keepdims=True) / m
    dp = np.dot(da, W)   # dp即损失函数对隐藏层激活值的偏导

    dbeta = np.sum(dp * p_ * (-x_c), axis=0, keepdims=True) / m

    assert dbeta.shape == beta.shape
    assert dw.shape == W.shape
    grad['dw'] = dw
    grad['dbeta'] = dbeta
    grad['db'] = db

    return grad

def compute_cost(y_hat_, y_):
    m = y_.shape[0]
    loss = np.sum((y_hat_ - y) ** 2) / (2 * m)
    return np.squeeze(loss)

def RBF_model(X_, y_, learning_rate, num_epochs, t):
    '''

    :param X_:
    :param y_:
    :param learning_rate:  学习率
    :param num_epochs:     迭代次数
    :param t:   隐藏层节点数量
    :return:
    '''
    parameters = {}
    np.random.seed(16)
    # 定义中心点，本来这里的中心点应该由随机采用或者聚类等非监督学习来获得的，这里为了简单就直接定义好了

    parameters['beta'] = np.random.randn(1, t)  # 初始化径向基的方差
    parameters['W'] = np.zeros((1, t))  # 初始化
    parameters['c'] = np.random.rand(t, 2)
    parameters['b'] = np.zeros([1, 1])
    costs = []

    for i in range(num_epochs):
        a, p, x_c = RBF_forward(X_, parameters)
        cost = compute_cost(a, y_)
        costs.append(cost)
        grad = RBF_backward(a, y_, x_c, p, parameters)

        parameters['beta'] -= learning_rate * grad['dbeta']
        parameters['W'] -= learning_rate * grad['dw']
        parameters['b'] -= learning_rate * grad['db']

    return parameters, costs


def predict(X_, parameters_):
    a, p, x_c = RBF_forward(X_, parameters_)

    return a

In [7]:
X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
y = np.array([[1], [1], [0], [0]])
#

parameters, costs = RBF_model(X, y, 0.003, 10000, 8)

plt.plot(costs)
plt.show()

print(predict(X, parameters))


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "D:\ProgramData\Anaconda3\envs\study_py37\lib\bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)

--Return--
None
> <ipython-input-7-8a45deadf108>(2)<module>()
      1 import pdb
----> 2 pdb.set_trace()
      3 X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
      4 y = np.array([[1], [1], [0], [0]])
      5 #

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

> c:\users\fx2\appdata\roaming\python\python37\site-packages\ipython\core\interactiveshell.py(3349)run_ast_nodes()
   3347                     to_run.append((node, 'single'))
   3348 
-> 3349          

BdbQuit: 